In [124]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [125]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [126]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight).

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [127]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [128]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [129]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [130]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [131]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [132]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [133]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [134]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [135]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [136]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [137]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [138]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [139]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [140]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [141]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [142]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [143]:
Level=8;

### Calculate the Proficiency Bonus

In [144]:
ProficiencyBonus=1+np.ceil(Level/4)

In [145]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [146]:
OtherBonus=1; #Rod of pact keeper

In [147]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [148]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [149]:
WeaponBonus=1; #Magical +1 weapon

In [150]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [151]:
MeleeHit

8

In [152]:
### Calculate Hit for Weapon Attacks with Dexterity

In [153]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [154]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [155]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [156]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [157]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [158]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [159]:
PrMiss=(OAC- SpellHit -1)/20 

In [160]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [161]:
Ad=2

In [162]:
PrHitA=1-(PrMiss)**(Ad)

In [163]:
PrHitA

0.91

#### testing function

In [164]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [165]:
Beams=min(math.floor(Level/5)+1,4)

In [166]:
Beams

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [167]:
AverageAgonisingBlastDamage=5.5

In [168]:
AverageDamagePerBeam=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [169]:
AverageDamagePerBeam

9.5

Additional modifier for Hex spell

In [170]:
HexAverageDamage=3.5

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [171]:
Critit=1/20;

In [172]:
2 * PrHit * PrMiss * AverageDamagePerBeam+ 2* (Critit) *(1-Critit) * AverageDamagePerBeam+ PrHit**2 * 2* AverageDamagePerBeam + 2* (Critit) *(Critit) * AverageDamagePerBeam

14.249999999999998

In order to easily generalise to higher number of Beams we use the binomial distribution. we check for 2 Beams to see the same result as above.

Without taking into account Crit Hit

In [173]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerBeam + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerBeam 

In [174]:
ADNCR

13.3

Calculating Crit Hit

In [175]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerBeam + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerBeam 

In [176]:
ACHD

0.9500000000000003

Adding the two together

In [177]:
TAD=ADNCR+ACHD

In [178]:
TAD

14.250000000000002

#### Calculating for arbitrary number of Beams

In [179]:
TotalDamage=0;

In [180]:
indexBeam = 1
while indexBeam < (Beams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,Beams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,Beams,Critit)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [181]:
TotalDamage

14.25

### Average damage with  Advantage

In [182]:
n=1

In [183]:
TotalBeams=n*Beams

In [184]:
CrititA=1-(1-1/20)**2

In [185]:
TotalDamage=0

In [186]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [187]:
TotalDamage

19.142500000000005

### Average damage with Advantage and Hex

In [188]:
n=1

In [189]:
TotalBeams=n*Beams

In [190]:
TotalDamage=0

In [191]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)
    indexBeam += 1

In [192]:
TotalDamage

26.195000000000004

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [193]:
n=2;

In [194]:
TotalBeams=n*Beams

In [195]:
TotalDamage=0

In [196]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,Critit)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [197]:
TotalDamage

28.500000000000007

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [198]:
n=2;

In [199]:
TotalBeams=n*Beams

In [200]:
TotalDamage=0

In [201]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [202]:
  TotalDamage

38.285000000000004

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [203]:
n=2;

In [204]:
TotalBeams=n*Beams

In [205]:
TotalBeams

4

In [206]:
TotalDamage=0

In [207]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)
    indexBeam += 1

In [208]:
TotalDamage

52.39

#### Testing the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [209]:
repetitions=4;average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [210]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

52.39

### Without Action surge, only advantage with two eldrich blast rays (repetitions=2)

In [211]:
repetitions=Beams;average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [212]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

26.195

### With Action Surge and spirit shroud so that I can use the celestial warlock's bonus to radient damage.

In [213]:
spirit_shroud=4.5

In [214]:
radient_damage_bonus=Modifiers[Abilities.index('CHA')]# since it is applied only to the first beam, multiply with factor 1/2 to calculate average damage

In [215]:
repetitions=4; average_damage_per_hit=AverageDamagePerBeam+spirit_shroud++HexAverageDamage+radient_damage_bonus/2;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [216]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

78.585

#### Assuming fiend warlock (no radient bonus, but darkness as a bonus spell). Hence I have one spell slot to use for Hex 

In [217]:
repetitions=4; average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage+spirit_shroud;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [218]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

70.525

## Paladin built: calculate the damage of Melee (+1 weapon) + Hex+ Green Flame Blade+ Radient damage bonus + two Smites with Advantage

#### Magical +1 Long sword damage 1d8+1 (+4) + Green flame blade (calculating for 2 targets) 

In [219]:
WeaponAverageDamage=4.5+1+Modifiers[Abilities.index('STR')]+2*4.5+4

In [220]:
WeaponAverageDamage

22.5

In [221]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [222]:
PrHit

0.7

In [223]:
PrMiss=1-PrHit

In [224]:
Ad=2

In [225]:
PrHitAd=1-(PrMiss)**(Ad)

In [226]:
PrHitAd

0.9099999999999999

#### Divine smite (3rd level slot) 4d8 average 4*4.5

In [227]:
Smite=4.5*4

#### bonus smite: I am assuming  searing smite and a celestial warlock's bonus to radient damage

In [228]:
BonusSmite=3*3.5+Modifiers[Abilities.index('CHA')]

In [229]:
HexAverageDamage=3.5 # I will cast it with a 1st level paladine slot

In [230]:
TotalDamage= (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [231]:
TotalDamage

58.93874999999999

#### Using the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [232]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [233]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

58.93875

#### assuming non-celestial warlock

In [234]:
BonusSmite=3*3.5

In [235]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [236]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

54.90875

## Level 8. Higher than the fighter (when Eldrich Blast only is considered) but asks for the consumption of two spell slots of the highest level (+1 first level) plus one 1st level (Hex). The characters casts first searing smite as bonus action. If the Fighter combo consumes one slot with spirit shroud (radient) and one with Hex then I can exploit the Celestial Warlock Bonus and surpass the Paladin combo but I have utilised all slots. At this level, I would not have another high level slot as a Paladin to also cast spirit shroud. The fighter combo suprasses the paladin combo also without the Celestial Warlock Bonus. 

# Converting code to OOP / Checks

In [64]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [65]:
import importlib 

In [66]:
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

In [67]:
Promytheas=dndopt.Character(19, 14, 12, 16, 8, 18)

In [68]:
Promytheas.set_level(8)

In [69]:
Promytheas.level

8

In [70]:
Promytheas.return_ability_score('STR')

19

In [71]:
Promytheas.return_ability_score('CHA')

18

In [72]:
Promytheas.set_eldrich_blast(1,10,1,1)

In [73]:
Promytheas.eldrich_blast_mean_damage

9.5

In [74]:
Promytheas.eldirch_blast_hit

8.0

In [75]:
Promytheas.set_advantages(1,0,0,0)

In [76]:
Promytheas.hit_opponent_probability(15,Promytheas.eldirch_blast_hit)

0.91

In [77]:
Promytheas.total_average_damage(15,Promytheas.eldirch_blast_hit,2,Promytheas.eldrich_blast_mean_damage)

19.1425

In [78]:
Promytheas.set_hex(1)

In [79]:
Promytheas.hex

3.5

In [80]:
Promytheas.total_average_damage(15,Promytheas.eldirch_blast_hit,2,Promytheas.eldrich_blast_mean_damage+Promytheas.hex)

26.195

In [81]:
Promytheas.total_average_damage(15,Promytheas.eldirch_blast_hit,4,Promytheas.eldrich_blast_mean_damage+Promytheas.hex)

52.39